# Introduction & Overview

## Yes, the stock market

This notebook provides an analysis and approach to stock-market activities. Every day the market is open, the question to the would-be trader presents:
- Buy
- Sell
- Hold

This notebook explores several approaches to answering this *daily* question.

*daily*? Modern algorithmic trading asks not daily, but [many times per second](https://en.wikipedia.org/wiki/High-frequency_trading), whether to buy, sell, or hold. Being a lay practitioner without access to industrial trading tools, I approach this as a per-day question.

## The CRISP-DM Framework

The process is described according to the [CRISP-DM framework](https://www.datascience-pm.com/crisp-dm-2/), a standardized process for a data-mining project such as this one. All quotes that follow,
> which are indicated like this

are from the paper titled *CRISP-DM: Towards a Standard Process Model for Data Mining* by Wirth & Hipp, in the document "crisp-dm-overview.pdf" as downloaded [from this source](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip), and copied [locally here](./docs/crisp-dm-overview.pdf).

# Imports

In [2]:
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
plt.rcParams.update(
    {
        "axes.grid": True,
        "figure.figsize": (16, 4),
    },
)

# Business Understanding

> This initial phase focuses on understanding the project objectives and requirements from a business perspective, and then converting this knowledge into a data mining problem definition, and a preliminary project plan designed to achieve the objectives.

# Data Understanding

> The data understanding phase starts with an initial data collection and proceeds with activities in order to get familiar with the data, to identify data quality problems, to discover first insights into the data, or to detect interesting subsets to form hypotheses for hidden information.

# Data Preparation

> The data preparation phase covers all activities to construct the final dataset (data that will be fed into the modeling tool(s)) from the initial raw data. Data preparation tasks are likely to be performed multiple times, and not in any prescribed order. Tasks include table, record, and attribute selection, data cleaning, construction of new attributes, and transformation of data for modeling tools.

# Modeling

> In this phase, various modeling techniques are selected and applied, and their parameters are calibrated to optimal values. Typically, there are several techniques for the same data mining problem type. Some techniques require specific data formats.

# Evaluation

> At this stage in the project you have built one or more models that appear to have high quality, from a data analysis perspective. Before proceeding to final deployment of the model, it is important to more thoroughly evaluate the model, and review the steps executed to construct the model, to be certain it properly achieves the business objectives. A key objective is to determine if there is some important business issue that has not been sufficiently considered. At the end of this phase, a decision on the use of the data mining results should be reached.

# Deployment

> Creation of the model is generally not the end of the project. Usually, the knowledge gained will need to be organized and presented in a way that the customer can use it. Depending on the requirements, the deployment phase can be as simple as generating a report or as complex as implementing a repeatable data mining process. In many cases it will be the user, not the data analyst, who will carry out the deployment steps. In any case, it is important to understand up front what actions will need to be carried out in order to actually make use of the created models.